In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "BiasedMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            'per_task_examples': np.inf,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'lambda': 10,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:6' if torch.cuda.is_available() else 'cpu'), }

#     trial_id = str(uuid.uuid4())
    trial_id = f"seed={params['seed']}_epoch={params['epochs_per_task']}q__"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/seed=1_epoch=1q__


In [2]:
from datasets import BiasedMNIST
from trainers.fair_trainer import FairContinualTrainer
from metrics.fair_metric_manager import FairMetricCollector
from algorithms.sensitive import Heuristic2

if  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = min(params['per_task_examples'], 15000))

    n_feature = 3*28*28
    params['alpha'] = 0.005
    params['lambda'] = 50


noise = 0.0
params['alpha'] = 0.05


[0 1 2 3 4 5 6 7 8 9]


ValueError: Too many dimensions: 3 > 2.

In [ ]:
backbone = cl.backbones.MLP2Layers(input_dim=n_feature, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

# from trainers.FairContinualTrainer import FairContinualTrainer
# trainer = FairContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])

trainer = FairContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

NameError: name 'n_feature' is not defined